GPU configuration

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 795289409077233753
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14385217536
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7554617034109114663
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [4]:
!cat /proc/meminfo

MemTotal:       13297200 kB
MemFree:         9449240 kB
MemAvailable:   11084696 kB
Buffers:           79020 kB
Cached:          1765896 kB
SwapCached:            0 kB
Active:           401344 kB
Inactive:        3082940 kB
Active(anon):       3824 kB
Inactive(anon):  1643084 kB
Active(file):     397520 kB
Inactive(file):  1439856 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               324 kB
Writeback:             0 kB
AnonPages:       1639476 kB
Mapped:           636676 kB
Shmem:             11592 kB
KReclaimable:      79060 kB
Slab:             123508 kB
SReclaimable:      79060 kB
SUnreclaim:        44448 kB
KernelStack:        5600 kB
PageTables:        21124 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6648600 kB
Committed_AS:    4338052 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       78032 kB
VmallocChunk:          0 kB
Percpu:          

In [5]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Gen RAM Free: 11.3 GB  | Proc size: 1.3 GB
GPU RAM Free: 14768MB | Used: 333MB | Util   2% | Total 15360MB


In [6]:
# Do not run unless the utilization is greater than 0%
# !kill -9 -1

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import time

X, y = joblib.load('/content/drive/MyDrive/DATASETS/OPIM04/small_dataset.pkl')

In [9]:
from sklearn.decomposition import PCA

init_time = time.time()
pca_obj = PCA(n_components=0.95)
pca_obj.fit(X)
print(pca_obj.explained_variance_)
end_time = time.time()
print("{:.2f} segundos transcurridos.".format(end_time-init_time))

[47912556.19832166 15601955.0980269   9560696.31828067  8677593.67855733
  5216893.00403294  4684754.85619269  2160782.98305553  1363200.1143105
  1241280.46653418   861725.21943555   771382.717045     678673.47905487
   546454.28275696   509058.57050548   474185.27223271   457867.4627369
   368501.17324673   324528.08020177   308518.88631308   300857.57642702
   270077.35680966   238294.71182284   226386.52900048   217945.11133877
   206391.90835713   190066.76625734   169082.17368019   160927.48743052
   153665.642408     148549.37900936   142793.35581509   131953.13116774
   124446.4551192    122963.22803315   114216.38509685   108312.11781024
   101668.04554978    95789.48397894    92148.8266492     88968.89531717
    81965.53633403    81564.38470887]
335.70 segundos transcurridos.


In [10]:
min_scaler = MinMaxScaler()
min_scaler.fit(X)
X_mscaler = min_scaler.transform(X)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(X_mscaler,
                                                    y,
                                                    test_size=0.20,
                                                    stratify= y)

In [12]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
import numpy as np

In [13]:
# Convert labels to categorical one-hot encoding
train_labels = to_categorical(y_train)
test_labels = to_categorical(y_test)

In [13]:
network = Sequential()
network.add(Dense(units=1024, activation="relu", input_dim=97200))
network.add(Flatten())
network.add(Dense(units=512, activation="relu"))
network.add(Dense(units=256, activation="relu"))
network.add(Dense(units=5, activation="softmax"))
network.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 1024)              99533824  
                                                                 
 flatten_1 (Flatten)         (None, 1024)              0         
                                                                 
 dense_4 (Dense)             (None, 512)               524800    
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 dense_6 (Dense)             (None, 5)                 1285      
                                                                 
Total params: 100,191,237
Trainable params: 100,191,237
Non-trainable params: 0
_________________________________________________________________


In [ ]:
network.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])
network.fit(x_train, train_labels, batch_size=128, epochs=50)
network.evaluate(x_test, test_labels, batch_size=128)

In [12]:
# Define the model architecture
#model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(input_shape[0], input_shape[1], 1)))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
'''
model = Sequential()
model.add(Dense(units=128, input_dim=97200))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='softmax'))
'''

In [13]:
# Compile the model
'''
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
'''

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
# Train the model
'''
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
'''

In [ ]:
# Evaluate the model
'''
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
'''